## Transformata Fouriera i filtracja przy pomocy maski
Celem zadania jest zaimplementowanie filtracji obrazu przy pomocy maski i transformaty Fouriera. 
Aby tego dokonać należy:
1. Przejrzeć część wykładku dotyczącego transformaty Fouriera i filtracji w dziedzinie częstotliwości.
2. Znaleźć rozwiązanie na slajdzie.
3. Zaimplementować rozwiazanie w odpowiednim miejscu w kodzie.
4. Zmierzyć czas wykonania operacji.

Filtracja odbywa się w dziedzinie częstotliwości więc:

x = F(img) - transformata Fouriera obrazu img

m = F(mask) - transformata Fouriera maski. Jak to zrobić zostało wyjaśnione na wykładzie.

x' = x * m - mnożenie punktowe

y = F^-1(x') - odwrotna transformata Fouriera


### Dodatkowo
Zaimplementuj funkcję `create_fft_mask` zgodnie z opisem znajdującym się na wykładzie.

Zaimplementują funkcję `apply_mask_with_fft` zgodnie z opisem znajdującym się na wykładzie.

Zmierz czas wykonywania funkcji `apply_mask_with_fft` i `apply_mask_with_convole` dla danego obrazu i maski składającej się z samych jedynek. Zbadaj czas operacji dla masek o rozmiarze {3, 9, 27, 81}. Kiedy bardziej opłaca się użyć transformaty Fouriera?

In [14]:
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from scipy import ndimage
import time


def create_fft_mask(mask: np.ndarray, shape: Tuple[int, int]) -> np.ndarray:
    h, w = mask.shape
    H = shape[0]
    W = shape[1]
    mask = np.ones((h, w), dtype=np.float32)
    padded_mask = np.zeros((H, W), dtype=np.float32)
    padded_mask[:h, :w] = mask
    padded_mask = np.fft.ifftshift(padded_mask)
    fft_mask = np.fft.fft2(padded_mask)
    return fft_mask


def apply_mask_with_convolve(image: np.ndarray, mask: np.ndarray) -> np.ndarray:
    start_time = time.time()
    to_return = ndimage.convolve(image, mask, mode='constant', cval=0.0)
    stop_time = time.time()
    return  to_return, (stop_time - start_time)


def apply_mask_with_fft(image: np.ndarray, mask: np.ndarray) -> np.ndarray:
    fft_mask = create_fft_mask(mask, image.shape)
    start_time = time.time()
    f_img = np.fft.fft2(image)
    f_result = f_img * fft_mask
    filtered_img = np.fft.ifft2(f_result)
    filtered_img = np.log1p(np.abs(filtered_img))

    end_time = time.time()
    elapsed = end_time - start_time
    return filtered_img, elapsed


if __name__ == '__main__':
    image = io.imread('bardzo-duzy-obraz.jpg', as_gray=True).astype(np.float32)
    mask_sizes = [3, 9, 27, 81]

    print(f"{'Maska':>6} | {'FFT [s]':>10} | {'Convolve [s]':>12}")
    print("-" * 36)

    for size in mask_sizes:
        _, t_fft = apply_mask_with_fft(image, np.ones((size, size), dtype=np.float32))
        _, t_conv = apply_mask_with_convolve(image, np.ones((size, size), dtype=np.float32))
        print(f"{size:6} | {t_fft:10.6f} | {t_conv:12.6f}")


 Maska |    FFT [s] | Convolve [s]
------------------------------------
     3 |   2.830762 |     0.233194
     9 |   3.004095 |     1.702240
    27 |   2.766222 |    16.254865
    81 |   2.787148 |   150.845940


Transformata Fouriera jest bardziej wydajna dla większych masek jednakże dla małych jest wolniejsza od ndimage.convolve